In [2]:
from datasets import load_dataset
import pandas as pd

train_dataset = load_dataset("swan07/authorship-verification", split="train[:5000]")
test_dataset = load_dataset("swan07/authorship-verification", split="test[:1000]")

from specter import SpecterVectorizer

spectorVectorizer = SpecterVectorizer()
spectorVectorizer.load()

train_spector = spectorVectorizer.embed(train_dataset)
test_spector = spectorVectorizer.embed(test_dataset)

from bert import BertVectorizer

bertVectorizer = BertVectorizer()
bertVectorizer.load()

train_bert = bertVectorizer.embed(train_dataset)
test_bert = bertVectorizer.embed(test_dataset)



/Users/1kl1/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from ngram import NgramCountVectorizer

ngramVectorizer = NgramCountVectorizer()
# ngramVectorizer.train(train_dataset)
# ngramVectorizer.save("./models/train_ngram_model.pkl")
ngramVectorizer.load("./models/pretrained_ngram_model.pkl")
train_ngram = ngramVectorizer.embed(train_dataset)
test_ngram = ngramVectorizer.embed(test_dataset)


from tf_idf import TfIdfVectorizer
tfidfVectorizer = TfIdfVectorizer()
# tfidfVectorizer.train(train_dataset)
# tfidfVectorizer.save("./models/train_tfidf_model.pkl")
tfidfVectorizer.load("./models/pretrained_tfidf_model.pkl")
train_tfidf = tfidfVectorizer.embed(train_dataset)
test_tfidf = tfidfVectorizer.embed(test_dataset)

Vectorizer가 './models/pretrained_tfidf_model.pkl'에서 로드되었습니다.


In [ ]:
from pathlib import Path
import pickle

Path("./embeddings").mkdir(exist_ok=True)

# 저장
print("저장 중...")
with open('./embeddings/train_specter.pkl', 'wb') as f:
    pickle.dump(train_spector, f)
print("✓ train_specter.pkl")

with open('./embeddings/test_specter.pkl', 'wb') as f:
    pickle.dump(test_spector, f)
print("✓ test_specter.pkl")

with open('./embeddings/train_bert.pkl', 'wb') as f:
    pickle.dump(train_bert, f)
print("✓ train_bert.pkl")

with open('./embeddings/test_bert.pkl', 'wb') as f:
    pickle.dump(test_bert, f)
print("✓ test_bert.pkl")

with open('./embeddings/train_ngram.pkl', 'wb') as f:
    pickle.dump(train_ngram, f)
print("✓ train_ngram.pkl")

with open('./embeddings/test_ngram.pkl', 'wb') as f:
    pickle.dump(test_ngram, f)
print("✓ test_ngram.pkl")

with open('./embeddings/train_tfidf.pkl', 'wb') as f:
    pickle.dump(train_tfidf, f)
print("✓ train_tfidf.pkl")

with open('./embeddings/test_tfidf.pkl', 'wb') as f:
    pickle.dump(test_tfidf, f)
print("✓ test_tfidf.pkl")


저장 중...
✓ train_ngram.pkl
✓ test_ngram.pkl
✓ train_tfidf.pkl
✓ test_tfidf.pkl


In [1]:
from pathlib import Path
import pickle


with open('./embeddings/train_specter.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('./embeddings/test_specter.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [2]:
from classifier import train_authorship_classifiers
models, results, train_preds_df, test_preds_df = train_authorship_classifiers(train_data, test_data, True)

저자 동일성 분류 모델 학습 시작

학습 데이터 크기: 5000 samples
테스트 데이터 크기: 1000 samples

[Train] 클래스 분포 - 같은 저자: 2411, 다른 저자: 2589
[Test] 클래스 분포 - 같은 저자: 492, 다른 저자: 508

Feature 준비 중...

사용 디바이스: cuda

1. 코사인 유사도 분류기 학습
최적 Threshold: 0.8800
Train Accuracy: 0.5960 | Test Accuracy: 0.6170

2. SVM 분류기 학습
Train Accuracy: 0.9036 | Test Accuracy: 0.6380

3. Random Forest 분류기 학습
Train Accuracy: 1.0000 | Test Accuracy: 0.6170

4. Logistic Regression 분류기 학습
Train Accuracy: 0.9984 | Test Accuracy: 0.5740

5. Gaussian Naive Bayes 분류기 학습
Train Accuracy: 0.6238 | Test Accuracy: 0.6270

6. Simple Neural Network 학습
Epoch [10/50], Loss: 0.1976, Test Acc: 0.6360
Epoch [20/50], Loss: 0.1109, Test Acc: 0.6270
Epoch [30/50], Loss: 0.0481, Test Acc: 0.6220
Epoch [40/50], Loss: 0.0374, Test Acc: 0.6150
Epoch [50/50], Loss: 0.0073, Test Acc: 0.6360
Train Accuracy: 1.0000 | Test Accuracy: 0.6360

7. Deep Neural Network 학습
Epoch [10/50], Loss: 0.2686, Test Acc: 0.6300
Epoch [20/50], Loss: 0.1227, Test Acc: 0.5950
Epoch [30/50],

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    matthews_corrcoef
)
from typing import Dict, List


class MultiModelEvaluator:
    """여러 모델의 예측 결과를 평가하는 클래스"""

    def __init__(self, pred_columns: List[str] = None):
        """
        Args:
            pred_columns: 평가할 예측 컬럼 리스트 (None이면 자동 감지)
        """
        self.pred_columns = pred_columns

    def _get_pred_columns(self, df: pd.DataFrame) -> List[str]:
        """예측 컬럼 자동 감지"""
        if self.pred_columns is not None:
            return self.pred_columns

        # 'pred_'로 시작하는 컬럼 찾기
        pred_cols = [col for col in df.columns if col.startswith('pred_')]
        if not pred_cols:
            raise ValueError("예측 컬럼을 찾을 수 없습니다. 'pred_'로 시작하는 컬럼이 필요합니다.")

        return pred_cols

    def evaluate_single_model(self, y_true: np.ndarray, y_pred: np.ndarray,
                            model_name: str = "") -> Dict:
        """
        단일 모델 평가

        Args:
            y_true: 실제 레이블
            y_pred: 예측값
            model_name: 모델 이름

        Returns:
            평가 지표 딕셔너리
        """
        metrics = {
            'model': model_name,
            'n_samples': len(y_true),
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1_score': f1_score(y_true, y_pred, zero_division=0),
            'mcc': matthews_corrcoef(y_true, y_pred)
        }

        # Confusion Matrix
        cm = confusion_matrix(y_true, y_pred)
        metrics['confusion_matrix'] = {
            'tn': int(cm[0, 0]),
            'fp': int(cm[0, 1]),
            'fn': int(cm[1, 0]),
            'tp': int(cm[1, 1])
        }

        # Specificity와 기타 지표
        tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
        metrics['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0
        metrics['sensitivity'] = tp / (tp + fn) if (tp + fn) > 0 else 0  # = recall

        return metrics

    def evaluate_all_models(self, df: pd.DataFrame,
                          true_label_col: str = 'true_label') -> pd.DataFrame:
        """
        모든 모델 평가

        Args:
            df: 예측 결과가 포함된 데이터프레임
            true_label_col: 실제 레이블 컬럼명

        Returns:
            모든 모델의 평가 지표를 담은 DataFrame
        """
        if true_label_col not in df.columns:
            raise ValueError(f"'{true_label_col}' 컬럼이 없습니다.")

        pred_cols = self._get_pred_columns(df)
        y_true = df[true_label_col].values

        results = []
        for pred_col in pred_cols:
            y_pred = df[pred_col].values
            model_name = pred_col.replace('pred_', '')

            metrics = self.evaluate_single_model(y_true, y_pred, model_name)
            results.append(metrics)

        # Confusion matrix 제외하고 DataFrame 생성
        results_simple = []
        for r in results:
            simple = {k: v for k, v in r.items() if k != 'confusion_matrix'}
            results_simple.append(simple)

        results_df = pd.DataFrame(results_simple)

        # 정렬 (accuracy 기준 내림차순)
        results_df = results_df.sort_values('accuracy', ascending=False).reset_index(drop=True)

        return results_df, results

    def print_report(self, df: pd.DataFrame, true_label_col: str = 'true_label'):
        """
        평가 결과를 보기 좋게 출력

        Args:
            df: 평가할 데이터프레임
            true_label_col: 실제 레이블 컬럼명
        """
        results_df, full_results = self.evaluate_all_models(df, true_label_col)

        print(f"\n{'='*100}")
        print(f"📊 모델 평가 결과 ({len(df):,} samples)")
        print(f"{'='*100}")

        # 메인 테이블
        print(f"\n{'Model':<20} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} "
              f"{'F1-Score':>10} {'MCC':>10}")
        print("-" * 100)

        for _, row in results_df.iterrows():
            print(f"{row['model']:<20} {row['accuracy']:>10.4f} {row['precision']:>10.4f} "
                  f"{row['recall']:>10.4f} {row['f1_score']:>10.4f} {row['mcc']:>10.4f}")

        print("-" * 100)

        # 최고 성능 모델
        best_model = results_df.iloc[0]
        print(f"\n🏆 최고 성능 모델: {best_model['model']}")
        print(f"   Accuracy: {best_model['accuracy']:.4f}")
        print(f"   F1-Score: {best_model['f1_score']:.4f}")

        # 상세 Confusion Matrix (상위 3개 모델)
        print(f"\n📋 상위 3개 모델 상세 정보:")
        print("=" * 100)

        for i in range(min(3, len(full_results))):
            result = full_results[i]
            cm = result['confusion_matrix']

            print(f"\n{i+1}. {result['model'].upper()}")
            print(f"   Accuracy: {result['accuracy']:.4f} | Precision: {result['precision']:.4f} | "
                  f"Recall: {result['recall']:.4f} | F1: {result['f1_score']:.4f}")
            print(f"\n   Confusion Matrix:")
            print(f"              예측")
            print(f"           0     1")
            print(f"   실제 0  {cm['tn']:4d}  {cm['fp']:4d}")
            print(f"        1  {cm['fn']:4d}  {cm['tp']:4d}")

        print(f"\n{'='*100}\n")

    def compare_models(self, df: pd.DataFrame, true_label_col: str = 'true_label',
                      metric: str = 'accuracy') -> pd.DataFrame:
        """
        특정 지표 기준으로 모델 비교

        Args:
            df: 예측 결과 DataFrame
            true_label_col: 실제 레이블 컬럼명
            metric: 비교할 지표 ('accuracy', 'f1_score', 'precision', 'recall', 'mcc')

        Returns:
            지표 기준 정렬된 DataFrame
        """
        results_df, _ = self.evaluate_all_models(df, true_label_col)

        if metric not in results_df.columns:
            raise ValueError(f"'{metric}' 지표가 없습니다. "
                           f"가능한 지표: {results_df.columns.tolist()}")

        return results_df.sort_values(metric, ascending=False).reset_index(drop=True)

    def get_ensemble_prediction(self, df: pd.DataFrame, method: str = 'majority') -> pd.Series:
        """
        앙상블 예측 생성

        Args:
            df: 예측 결과 DataFrame
            method: 'majority' (다수결) 또는 'unanimous' (만장일치)

        Returns:
            앙상블 예측 Series
        """
        pred_cols = self._get_pred_columns(df)

        if method == 'majority':
            # 다수결 투표
            ensemble = df[pred_cols].mode(axis=1)[0]
        elif method == 'unanimous':
            # 모든 모델이 동의하는 경우만 1
            ensemble = df[pred_cols].all(axis=1).astype(int)
        else:
            raise ValueError(f"Unknown method: {method}")

        return ensemble

    def analyze_disagreements(self, df: pd.DataFrame,
                             true_label_col: str = 'true_label') -> pd.DataFrame:
        """
        모델 간 의견 불일치 분석

        Args:
            df: 예측 결과 DataFrame
            true_label_col: 실제 레이블 컬럼명

        Returns:
            의견 불일치 통계 DataFrame
        """
        pred_cols = self._get_pred_columns(df)

        # 각 샘플에 대해 예측이 일치하지 않은 정도 계산
        df_copy = df.copy()
        df_copy['n_agree'] = df_copy[pred_cols].sum(axis=1)  # 1로 예측한 모델 수
        df_copy['all_agree'] = (df_copy['n_agree'] == 0) | (df_copy['n_agree'] == len(pred_cols))
        df_copy['disagreement'] = ~df_copy['all_agree']

        # 통계
        stats = {
            'total_samples': len(df),
            'all_agree': df_copy['all_agree'].sum(),
            'disagreement': df_copy['disagreement'].sum(),
            'disagreement_rate': df_copy['disagreement'].mean()
        }

        # 의견 불일치 샘플 중 정답률
        if stats['disagreement'] > 0:
            disagreement_samples = df_copy[df_copy['disagreement']]
            ensemble_pred = self.get_ensemble_prediction(disagreement_samples, method='majority')
            stats['disagreement_accuracy'] = accuracy_score(
                disagreement_samples[true_label_col],
                ensemble_pred
            )

        print(f"\n{'='*60}")
        print(f"🔍 모델 간 의견 불일치 분석")
        print(f"{'='*60}")
        print(f"전체 샘플: {stats['total_samples']:,}")
        print(f"전원 일치: {stats['all_agree']:,} ({stats['all_agree']/stats['total_samples']*100:.1f}%)")
        print(f"의견 불일치: {stats['disagreement']:,} ({stats['disagreement_rate']*100:.1f}%)")

        if 'disagreement_accuracy' in stats:
            print(f"의견 불일치 샘플의 앙상블 정확도: {stats['disagreement_accuracy']:.4f}")
        print(f"{'='*60}\n")

        return df_copy[df_copy['disagreement']]


def evaluate_predictions(train_preds_df: pd.DataFrame,
                        test_preds_df: pd.DataFrame,
                        true_label_col: str = 'true_label',
                        verbose: bool = True):
    """
    Train/Test 예측 결과 평가 메인 함수

    Args:
        train_preds_df: Train 예측 결과 DataFrame
        test_preds_df: Test 예측 결과 DataFrame
        true_label_col: 실제 레이블 컬럼명
        verbose: 상세 출력 여부

    Returns:
        train_results, test_results (각각 평가 결과 DataFrame)
    """
    evaluator = MultiModelEvaluator()

    # Train 평가
    print("\n" + "🎓 TRAIN SET 평가 " + "="*80)
    if verbose:
        evaluator.print_report(train_preds_df, true_label_col)
    train_results, _ = evaluator.evaluate_all_models(train_preds_df, true_label_col)

    # Test 평가
    print("\n" + "🧪 TEST SET 평가 " + "="*80)
    if verbose:
        evaluator.print_report(test_preds_df, true_label_col)
    test_results, _ = evaluator.evaluate_all_models(test_preds_df, true_label_col)

    # 비교
    print("\n" + "📊 TRAIN vs TEST 비교 " + "="*80)
    comparison = pd.merge(
        train_results[['model', 'accuracy', 'f1_score']],
        test_results[['model', 'accuracy', 'f1_score']],
        on='model',
        suffixes=('_train', '_test')
    )
    comparison['overfitting'] = comparison['accuracy_train'] - comparison['accuracy_test']
    comparison = comparison.sort_values('accuracy_test', ascending=False)

    print(f"\n{'Model':<20} {'Train Acc':>10} {'Test Acc':>10} {'Overfit':>10} {'Test F1':>10}")
    print("-" * 80)
    for _, row in comparison.iterrows():
        print(f"{row['model']:<20} {row['accuracy_train']:>10.4f} {row['accuracy_test']:>10.4f} "
              f"{row['overfitting']:>10.4f} {row['f1_score_test']:>10.4f}")
    print("-" * 80)

    return train_results, test_results, comparison


if __name__ == "__main__":
    evaluator = MultiModelEvaluator()
    evaluator.print_report(test_preds_df)

    # 특정 지표로 비교
    print("\nF1-Score 기준 모델 비교:")
    f1_comparison = evaluator.compare_models(test_preds_df, metric='f1_score')
    print(f1_comparison[['model', 'f1_score', 'accuracy']])

    # 앙상블 예측
    ensemble = evaluator.get_ensemble_prediction(test_preds_df, method='majority')
    print(f"\n앙상블 정확도: {accuracy_score(test_preds_df['true_label'], ensemble):.4f}")

    # 의견 불일치 분석
    disagreements = evaluator.analyze_disagreements(test_preds_df)
    print(f"의견 불일치 샘플 수: {len(disagreements)}")

사용 예시:
------------------------------------------------------------

📊 모델 평가 결과 (1,000 samples)

Model                  Accuracy  Precision     Recall   F1-Score        MCC
----------------------------------------------------------------------------------------------------
svm                      0.6380     0.6519     0.5671     0.6065     0.2766
simple_nn                0.6360     0.6356     0.6098     0.6224     0.2716
nb                       0.6270     0.6112     0.6646     0.6368     0.2558
deep_nn                  0.6180     0.6083     0.6280     0.6180     0.2363
cosine                   0.6170     0.6598     0.4573     0.5402     0.2415
rf                       0.6170     0.6589     0.4593     0.5413     0.2412
siamese_nn               0.5820     0.5654     0.6504     0.6049     0.1676
lr                       0.5740     0.5668     0.5691     0.5680     0.1478
----------------------------------------------------------------------------------------------------

🏆 최고 성능 모델: svm


In [11]:
evaluator